In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("data/chase_banking.pdf")
documents= loader.load()
print(documents)

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.5 (Macintosh)', 'creationdate': '2024-10-03T09:29:06-04:00', 'author': 'JPMorgan Chase Bank', 'keywords': 'Chase; total; checking; guide to your account; ada; (PDF)', 'moddate': '2024-10-07T09:59:35-04:00', 'subject': 'Chase Total Checking - A Guide To Your Account', 'title': 'Chase Total Checking - A Guide To Your Account (PDF)', 'trapped': '/Unknown', 'source': 'data/chase_banking.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='HAVE QUESTIONS? CALL US AT 1-800-935-9935 (WE ACCEPT OPERATOR RELAY CALLS)  • WANT MORE INFO?  SEE THE DEPOSIT ACCOUNT AGREEMENT\n1\nCHASE TOTAL CHECKING\n®\nA GUIDE TO YOUR ACCOUNT †\nIt’s important that you understand how your Chase Total Checking account works. \nWe’ve created this Guide to explain the fees and some key terms of your personal account.\nMONTHLY \nSERVICE FEE*\nMonthly Service Fee $12\nWays to Avoid the \nMonthly Service Fee\n$0 Monthl

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

Rec_splitter= RecursiveCharacterTextSplitter(
    separators =["\n\n","\n","."," ",""],
    chunk_size=100,
    chunk_overlap=20,
    
)

In [4]:
chunks = []
for doc in documents:
    texts = Rec_splitter.split_text(doc.page_content)
    chunks.extend(texts)
    
for chunk in enumerate(chunks):
    print(chunk)

(0, 'HAVE QUESTIONS? CALL US AT 1-800-935-9935 (WE ACCEPT OPERATOR RELAY CALLS)  • WANT MORE INFO?  SEE')
(1, 'MORE INFO?  SEE THE DEPOSIT ACCOUNT AGREEMENT')
(2, '1\nCHASE TOTAL CHECKING\n®\nA GUIDE TO YOUR ACCOUNT †')
(3, 'It’s important that you understand how your Chase Total Checking account works.')
(4, 'We’ve created this Guide to explain the fees and some key terms of your personal account.\nMONTHLY')
(5, 'MONTHLY \nSERVICE FEE*\nMonthly Service Fee $12\nWays to Avoid the \nMonthly Service Fee')
(6, 'Monthly Service Fee\n$0 Monthly Service Fee when you have any ONE  of the following during each')
(7, 'monthly statement period:')
(8, '•  Electronic deposits made into this account totaling $500 or more, such as')
(9, 'payments from payroll providers or government benefit providers, by using')
(10, '(i) the ACH network, (ii) the Real Time Payment or FedNow SM network, or  \n(ii')
(11, '(ii\ni) third party services that facilitate payments to your debit card using the')
(12, 'Visa®

1.QdrantDB

In [6]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

client= QdrantClient(url="http://localhost:6333")

client.recreate_collection(
    collection_name="index-SparseIndex-collection",
    vectors_config=models.VectorParams(size=1024, distance=models.Distance.DOT),
    
    sparse_vectors_config= {
        "text":models.SparseVectorParams(index=models.SparseIndexParams(on_disk=False))   
    },
    
    optimizers_config=models.OptimizersConfigDiff(default_segment_number=1)
)

C:\Users\sriva\AppData\Local\Temp\ipykernel_21176\922855453.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [7]:
import uuid
from openai import OpenAI
from qdrant_client.http.models import PointStruct

openai_client= OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

records=[]
model="mxbai-embed-large"

for index, chunk in enumerate(chunks):
    response=openai_client.embeddings.create(input=chunk, model=model)
    embeddings= response.data[0].embedding
    print(embeddings)
    
    record_id= str(uuid.uuid4())
    records.append(PointStruct(id=record_id,payload={"text":chunk},vector=embeddings))

[-0.0056892876, 0.010129904, -0.076591894, 0.017863262, 0.038715947, -0.025373545, 0.005930967, 0.026361566, 0.057150897, 0.0051120557, 0.0049930764, 0.04350821, -0.00461594, 0.00083553063, -0.040344164, -0.008594665, 0.00810301, -0.0108125815, 0.016239949, -0.025199056, -0.020593653, 0.012910188, -0.023561906, -0.039561853, -0.059274256, -0.025734533, 0.015286048, -0.044040658, 0.034612905, 0.040181905, -0.04511034, -0.005004059, 0.009285869, -0.021925619, -0.014623572, 0.0119793145, 0.030808734, -0.027339304, -0.061212383, -0.03497746, -0.042056076, 0.06590679, 0.048760105, -0.036351267, -0.04586698, 0.031634036, -0.05854596, -0.03007269, 0.06179782, -0.020386107, 0.013887826, 0.014801137, 0.015414612, 0.0029701958, 0.056469843, -0.030332517, -0.0019494696, -0.004820252, -0.0124337785, -0.0050366195, 0.0240572, -0.008285448, 0.0046746125, -0.031399254, 0.020694438, 0.034868397, -0.0075735264, 0.0126885, -0.03215622, -0.045574054, 0.010844097, 0.03909481, -0.05096583, 0.02989946, -0.0

In [8]:
client.upsert(
    collection_name="index-SparseIndex-collection",
    points=records,
    wait=True
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
question_vector= openai_client.embeddings.create(
    input='how to avoid overdraft',
    model=model
)

question_embeddings= question_vector.data[0].embedding
print(question_embeddings)

[-0.035258684, 0.037265968, -0.014393011, 0.008788542, 0.02625705, -0.026202343, 0.060541376, 0.0007688636, 0.024973532, 0.05166278, 0.045898017, 0.0020137324, 0.026556766, 0.020320842, -0.059252765, 0.06583778, -0.03908235, -0.009138738, -0.03317856, 0.028223755, -0.02329643, -0.045822818, -0.051633455, -0.063556746, -0.05503437, 0.037962154, 0.0039087995, -0.03616325, 0.09387711, 0.049975436, -0.030846056, -0.015270579, 0.012159656, 0.039331242, -0.05450934, -0.018848136, 0.019811107, -0.014107007, 0.0059596496, -0.062313586, -0.01475596, 0.021854091, 0.06403995, -0.0151218185, -0.04556501, 0.0051296353, -0.03321425, -0.012574183, 0.0026199284, -0.046330616, -0.006067385, 0.033480477, 0.039505642, 0.024242094, -0.02848557, 0.03164401, -0.0128364125, -0.042313144, -0.019618817, 0.03463664, -0.00323263, -0.009283769, 0.046781246, -0.046816237, 0.013065383, 0.06469377, -0.033420008, 0.020661358, 0.0022532246, 0.044330794, -0.007546882, 0.017432211, 0.0017069138, 0.020189857, -0.01053269

In [ ]:
response= client.search(collection_name="index-SparseIndex-collection",query_vector=question_embeddings)
print(response)

[ScoredPoint(id='8051310f-8a6b-41c7-91e1-15d9e988e47e', version=0, score=0.8405824, payload={'text': 'You can avoid overdrawing your account by making a deposit or transferring funds to cover the'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='621e171f-62d2-4e98-a3e5-8b60c8547116', version=0, score=0.8405824, payload={'text': 'You can avoid overdrawing your account by making a deposit or transferring funds to cover the'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='a46e0ff7-7641-408e-9c50-49e2a59ea907', version=0, score=0.81803304, payload={'text': 'funds to cover the overdraft before the'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='5d3a65f8-5568-41c3-a279-b7bbeec1acea', version=0, score=0.81803304, payload={'text': 'funds to cover the overdraft before the'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='2c5b9178-e4cc-4b2d-8e32-daf10cf78907', version=0, score=0.80432343, payload={'text': 'n\tthat\tcauses\tan

C:\Users\sriva\AppData\Local\Temp\ipykernel_21176\3318345871.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  response= client.search(collection_name="index-SparseIndex-collection",query_vector=question_embeddings)


In [12]:
for result in response:
    print(f"score:{result.score},response_text:{result.payload}")

score:0.8405824,response_text:{'text': 'You can avoid overdrawing your account by making a deposit or transferring funds to cover the'}
score:0.8405824,response_text:{'text': 'You can avoid overdrawing your account by making a deposit or transferring funds to cover the'}
score:0.81803304,response_text:{'text': 'funds to cover the overdraft before the'}
score:0.81803304,response_text:{'text': 'funds to cover the overdraft before the'}
score:0.80432343,response_text:{'text': 'n\tthat\tcauses\tan\toverdraft\tand\tcharge\tan\tOverdraft\tFee'}
score:0.8003143,response_text:{'text': 'bring your account balance to overdrawn by $50 or less at the end of the'}
score:0.79733753,response_text:{'text': 'Overdraft Protection transfers or transfers to maintain target balances in other'}
score:0.7894998,response_text:{'text': 'your account balance to overdrawn by $50 or less at the end of the next'}
score:0.7756997,response_text:{'text': 'on a business \nday when your \naccount balance is \noverdrawn